<a href="https://colab.research.google.com/github/Adam-Aber/EHA-Detect-Efficient-and-Hierarchical-Distributed-Architecture-for-Energy-Theft-Detection/blob/main/HFL_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages

import random
import math
import copy
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



def Creatattacks(d):
  alpha1 = random.uniform(0.1,0.2)
  at1 = np.dot(alpha1,d) #1st attack type
  minOffTime=20
  #minOffTime=8
  it = np.random.randint(0, high=47-minOffTime, size=d.shape[0])
  du = np.random.randint(minOffTime, high=48, size=d.shape[0])
  fl = np.minimum(it + du,48)
  at2 = copy.deepcopy(d)
  for l in np.arange(0,d.shape[0]):
    at2[l,it[l]:fl[l]]=0 #2nd attack type
  alpha2 = np.random.uniform(low=0.1, high=0.2, size=(d.shape[0],48))
  at3 = np.multiply(alpha2,d) # 3rd attack type
  avg = np.mean(d,axis=1) # customer daily average consumption
  avg_m = np.diag(avg) # in diagonal matrix for multiplication
  alpha3 = np.random.uniform(low=0.1, high=0.2, size=(d.shape[0],48))
  #alpha3 = np.random.uniform(low=0.1, high=0.5, size=(d.shape[0],48))
  at4 = np.dot(avg_m,alpha3) # 4th attack type
  alpha4 = np.ones((d.shape[0],48))
  at5 = np.dot(avg_m,alpha4) # 5th attack type
  at6 = np.fliplr(d) # 6th attack type
  return at1,at2,at3,at4,at5,at6

D= pd.read_csv('/content/data.csv', sep=',', header=None)
# print(D.shape)

customer=0
for i in range(D.shape[0]):
    benign=D.iloc[i,:]
    # print(benign.shape)
    benign=benign.to_numpy()

    benign = benign.reshape(536, 48)

    y_benign = np.zeros(536)


    X_train, X_test, y_train, y_test = train_test_split(benign, y_benign, test_size=0.33, random_state=42)
    num_samples_per_attack=math.ceil(X_test.shape[0]/6)
    # print(X_test.shape[0])
    # print(num_samples_per_attack)
    at1,at2,at3,at4,at5,at6 = Creatattacks(X_test[0:num_samples_per_attack,:])


    X_atta = np.concatenate((at1,at2,at3,at4,at5,at6), axis=0)
    y2 = np.ones(num_samples_per_attack*6)


    X = np.concatenate((X_test,X_atta), axis=0)
    Y = np.concatenate((y_test,y2), axis=0)

    with open('train_features%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, X_train, delimiter=",")

    with open('train_labels%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, y_train , delimiter=",")

    with open('test_benign_features%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, X_test, delimiter=",")

    with open('test_benign_labels%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, y_test , delimiter=",")

    with open('test_features%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, X, delimiter=",")

    with open('test_labels%s.csv'%customer,'a') as saved_file:
     np.savetxt(saved_file, Y, delimiter=",")



    with open('train_features.csv','a') as saved_file:
     np.savetxt(saved_file, X_train, delimiter=",")

    with open('train_labels.csv','a') as saved_file:
     np.savetxt(saved_file, y_train , delimiter=",")

    with open('test_benign_features.csv','a') as saved_file:
     np.savetxt(saved_file, X_test, delimiter=",")

    with open('test_benign_labels.csv','a') as saved_file:
     np.savetxt(saved_file, y_test , delimiter=",")

    with open('test_features.csv','a') as saved_file:
     np.savetxt(saved_file, X, delimiter=",")

    with open('test_labels.csv','a') as saved_file:
     np.savetxt(saved_file, Y, delimiter=",")

    customer+=1




In [ ]:
# Autoencoder
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot


def Build_Autoencoder():


  n_inputs=48
  # define encoder
  visible = Input(shape=(n_inputs,))
  # encoder level 0
  e = Dense(n_inputs*4)(visible)
  e = BatchNormalization()(e)
  e = LeakyReLU()(e)
  # encoder level 1
  e = Dense(n_inputs*2)(e)
  e = BatchNormalization()(e)
  e = LeakyReLU()(e)
  # encoder level 2
  e = Dense(n_inputs)(e)
  e = BatchNormalization()(e)
  e = LeakyReLU()(e)
  # bottleneck
  n_bottleneck = n_inputs
  bottleneck = Dense(n_bottleneck)(e)
  # define decoder, level 0
  d = Dense(n_inputs)(bottleneck)
  d = BatchNormalization()(d)
  d = LeakyReLU()(d)
  # define decoder, level 1
  d = Dense(n_inputs*2)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU()(d)
  # decoder level 2
  d = Dense(n_inputs*4)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(d)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  return model



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
comms_round = 20
lr = 0.01

# Create global model
global_model = Build_Autoencoder()
global_model.compile(optimizer='adam', loss='mse')

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Training loop
for comm_round in range(comms_round):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    if comm_round > 0:
        global_model.save(f'/content/model_round_{comm_round}.h5')
        global_model.save('/content/latest_model.h5')

    # evaluate the model
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

    preds = tf.math.less(clean, threshold)
    print(tf.math.count_nonzero(preds) / len(clean))
    print(tf.math.count_nonzero(preds))
    print(len(clean))

    TN = tf.math.count_nonzero(preds)
    FN = len(clean) - TN

    preds_a = tf.math.greater(fraud, threshold)
    print(tf.math.count_nonzero(preds_a)/len(fraud))
    print(tf.math.count_nonzero(preds_a))
    print(len(fraud))

    TP = tf.math.count_nonzero(preds_a)
    FP = len(fraud) - TP

    print((tf.math.count_nonzero(preds) / len(clean) + tf.math.count_nonzero(preds_a)/len(fraud))/2)

# Save final model
global_model.save('/content/final_model.h5')


Global Round: 0
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19
447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 46.75%
Precision: 47.22%
Recall: 47.61%
F1 Score: 47.41%
AUC: 0.4674
False Alarm Rate: 54.12%
Confusion Matrix:
[[3248 3832]
 [3772 3428]]
tf.Tensor(0.45875706214689266, shape=(), dtype=float64)
tf.Tensor(3248, shape=(), dtype=int64)
7080
tf.Tensor(0.4761111111111111, shape=(), dtype=float64)
tf.Tensor(3428, shape=(), dtype=int64)
7200
tf.Tensor(0.4674340866290019, shape=(), dtype=float64)

Global Round: 1
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 46.95%
Precision: 47.41%
Recall: 47.81%
F1 Score: 47.61%
AUC: 0.4694
False Alarm Rate: 53.93%
Confusion Matrix:
[[3262 3818]
 [3758 3442]]
tf.Tensor(0.46073446327683615, shape=(), dtype=float64)
tf.Tensor(3262, shape=(), dtype=int64)
7080
tf.Tensor(0.47805555555555557, shape=(), dtype=float64)
tf.Tensor(3442, shape=(), dtype=int64)
7200
tf.Tensor(0.46939500941619583, shape=(), dtype=float64)

Global Round: 2
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 55.22%
Precision: 55.55%
Recall: 56.01%
F1 Score: 55.78%
AUC: 0.5522
False Alarm Rate: 45.58%
Confusion Matrix:
[[3853 3227]
 [3167 4033]]
tf.Tensor(0.5442090395480226, shape=(), dtype=float64)
tf.Tensor(3853, shape=(), dtype=int64)
7080
tf.Tensor(0.5601388888888889, shape=(), dtype=float64)
tf.Tensor(4033, shape=(), dtype=int64)
7200
tf.Tensor(0.5521739642184558, shape=(), dtype=float64)

Global Round: 3
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 60.24%
Precision: 60.48%
Recall: 60.99%
F1 Score: 60.73%
AUC: 0.6023
False Alarm Rate: 40.52%
Confusion Matrix:
[[4211 2869]
 [2809 4391]]
tf.Tensor(0.5947740112994351, shape=(), dtype=float64)
tf.Tensor(4211, shape=(), dtype=int64)
7080
tf.Tensor(0.6098611111111111, shape=(), dtype=float64)
tf.Tensor(4391, shape=(), dtype=int64)
7200
tf.Tensor(0.6023175612052731, shape=(), dtype=float64)

Global Round: 4
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 64.27%
Precision: 64.45%
Recall: 64.99%
F1 Score: 64.72%
AUC: 0.6427
False Alarm Rate: 36.45%
Confusion Matrix:
[[4499 2581]
 [2521 4679]]
tf.Tensor(0.6354519774011299, shape=(), dtype=float64)
tf.Tensor(4499, shape=(), dtype=int64)
7080
tf.Tensor(0.6498611111111111, shape=(), dtype=float64)
tf.Tensor(4679, shape=(), dtype=int64)
7200
tf.Tensor(0.6426565442561205, shape=(), dtype=float64)

Global Round: 5
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 58.11%
Precision: 58.39%
Recall: 58.88%
F1 Score: 58.63%
AUC: 0.5810
False Alarm Rate: 42.67%
Confusion Matrix:
[[4059 3021]
 [2961 4239]]
tf.Tensor(0.5733050847457627, shape=(), dtype=float64)
tf.Tensor(4059, shape=(), dtype=int64)
7080
tf.Tensor(0.58875, shape=(), dtype=float64)
tf.Tensor(4239, shape=(), dtype=int64)
7200
tf.Tensor(0.5810275423728813, shape=(), dtype=float64)

Global Round: 17
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 56.99%
Precision: 57.29%
Recall: 57.76%
F1 Score: 57.52%
AUC: 0.5698
False Alarm Rate: 43.80%
Confusion Matrix:
[[3979 3101]
 [3041 4159]]
tf.Tensor(0.5620056497175141, shape=(), dtype=float64)
tf.Tensor(3979, shape=(), dtype=int64)
7080
tf.Tensor(0.5776388888888889, shape=(), dtype=float64)
tf.Tensor(4159, shape=(), dtype=int64)
7200
tf.Tensor(0.5698222693032016, shape=(), dtype=float64)

Global Round: 18
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy: 58.59%
Precision: 58.86%
Recall: 59.35%
F1 Score: 59.10%
AUC: 0.5858
False Alarm Rate: 42.19%
Confusion Matrix:
[[4093 2987]
 [2927 4273]]
tf.Tensor(0.5781073446327684, shape=(), dtype=float64)
tf.Tensor(4093, shape=(), dtype=int64)
7080
tf.Tensor(0.5934722222222222, shape=(), dtype=float64)
tf.Tensor(4273, shape=(), dtype=int64)
7200
tf.Tensor(0.5857897834274952, shape=(), dtype=float64)

Global Round: 19
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Accuracy: 56.92%
Precision: 57.22%
Recall: 57.69%
F1 Score: 57.46%
AUC: 0.5691
False Alarm Rate: 43.87%
Confusion Matrix:
[[3974 3106]
 [3046 4154]]
tf.Tensor(0.5612994350282486, shape=(), dtype=float64)
tf.Tensor(3974, shape=(), dtype=int64)
7080
tf.Tensor(0.5769444444444445, shape=(), dtype=float64)
tf.Tensor(4154, shape=(), dtype=int64)
7200
tf.Tensor(0.5691219397363465, shape=(), dtype=float64)


In [ ]:
# HFL 20 to 40

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
import os
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError as MSE

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
lr = 0.01

# Create metric instances for loading
mse_loss = MeanSquaredError()
mse_metric = MSE()

# Load the latest saved model
print("Loading latest model...")
global_model = load_model('/content/latest_model.h5',
                         custom_objects={
                             'MeanSquaredError': MeanSquaredError,
                             'mse': mse_loss,
                             'MSE': mse_metric
                         })

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Continue training
for comm_round in range(20, 40):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    global_model.save(f'/content/model_round_{comm_round}.h5')
    global_model.save('/content/latest_model.h5')

    # Model evaluation
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print(f"Round {comm_round} Results:")
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

# Save final model
global_model.save('/content/final_model.h5')

Loading latest model...



Global Round: 20
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 20 Results:
Accuracy: 51.74%
Precision: 52.12%
Recall: 52.56%
F1 Score: 52.34%
AUC: 0.5173
False Alarm Rate: 49.10%
Confusion Matrix:
[[3604 3476]
 [3416 3784]]

Global Round: 21
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 21 Results:
Accuracy: 49.93%
Precision: 50.34%
Recall: 50.76%
F1 Score: 50.55%
AUC: 0.4992
False Alarm Rate: 50.92%
Confusion Matrix:
[[3475 3605]
 [3545 3655]]

Global Round: 22
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 22 Results:
Accuracy: 53.87%
Precision: 54.21%
Recall: 54.67%
F1 Score: 54.44%
AUC: 0.5386
False Alarm Rate: 46.95%
Confusion Matrix:
[[3756 3324]
 [3264 3936]]

Global Round: 23
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 33 Results:
Accuracy: 71.44%
Precision: 71.50%
Recall: 72.10%
F1 Score: 71.80%
AUC: 0.7144
False Alarm Rate: 29.22%
Confusion Matrix:
[[5011 2069]
 [2009 5191]]

Global Round: 34
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 34 Results:
Accuracy: 69.92%
Precision: 70.00%
Recall: 70.58%
F1 Score: 70.29%
AUC: 0.6991
False Alarm Rate: 30.76%
Confusion Matrix:
[[4902 2178]
 [2118 5082]]

Global Round: 35
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 35 Results:
Accuracy: 64.24%
Precision: 64.42%
Recall: 64.96%
F1 Score: 64.69%
AUC: 0.6424
False Alarm Rate: 36.48%
Confusion Matrix:
[[4497 2583]
 [2523 4677]]

Global Round: 36
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 36 Results:
Accuracy: 70.66%
Precision: 70.73%
Recall: 71.32%
F1 Score: 71.02%
AUC: 0.7065
False Alarm Rate: 30.01%
Confusion Matrix:
[[4955 2125]
 [2065 5135]]

Global Round: 37
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 37 Results:
Accuracy: 72.58%
Precision: 72.62%
Recall: 73.22%
F1 Score: 72.92%
AUC: 0.7257
False Alarm Rate: 28.08%
Confusion Matrix:
[[5092 1988]
 [1928 5272]]

Global Round: 38
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9


In [ ]:
# HFL 20 to 40

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
import os
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError as MSE

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
lr = 0.01

# Create metric instances for loading
mse_loss = MeanSquaredError()
mse_metric = MSE()

# Load the latest saved model
print("Loading latest model...")
global_model = load_model('/content/latest_model.h5',
                         custom_objects={
                             'MeanSquaredError': MeanSquaredError,
                             'mse': mse_loss,
                             'MSE': mse_metric
                         })

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Continue training
for comm_round in range(20, 40):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    global_model.save(f'/content/model_round_{comm_round}.h5')
    global_model.save('/content/latest_model.h5')

    # Model evaluation
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print(f"Round {comm_round} Results:")
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

# Save final model
global_model.save('/content/final_model.h5')

Loading latest model...

Global Round: 20
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 20 Results:
Accuracy: 59.20%
Precision: 59.46%
Recall: 59.96%
F1 Score: 59.71%
AUC: 0.5920
False Alarm Rate: 41.57%
Confusion Matrix:
[[4137 2943]
 [2883 4317]]

Global Round: 21
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Round 21 Results:
Accuracy: 60.36%
Precision: 60.61%
Recall: 61.11%
F1 Score: 60.86%
AUC: 0.6036
False Alarm Rate: 40.40%
Confusion Matrix:
[[4220 2860]
 [2800 4400]]

Global Round: 22
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 22 Results:
Accuracy: 59.37%
Precision: 59.63%
Recall: 60.12%
F1 Score: 59.88%
AUC: 0.5936
False Alarm Rate: 41.40%
Confusion Matrix:
[[4149 2931]
 [2871 4329]]

Global Round: 23
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 23 Results:
Accuracy: 46.69%
Precision: 47.16%
Recall: 47.56%
F1 Score: 47.36%
AUC: 0.4669
False Alarm Rate: 54.18%
Confusion Matrix:
[[3244 3836]
 [3776 3424]]

Global Round: 24
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Round 32 Results:
Accuracy: 71.22%
Precision: 71.28%
Recall: 71.88%
F1 Score: 71.58%
AUC: 0.7121
False Alarm Rate: 29.45%
Confusion Matrix:
[[4995 2085]
 [2025 5175]]

Global Round: 33
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Round 33 Results:
Accuracy: 72.76%
Precision: 72.80%
Recall: 73.40%
F1 Score: 73.10%
AUC: 0.7275
False Alarm Rate: 27.90%
Confusion Matrix:
[[5105 1975]
 [1915 5285]]

Global Round: 34
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 34 Results:
Accuracy: 68.78%
Precision: 68.88%
Recall: 69.46%
F1 Score: 69.17%
AUC: 0.6878
False Alarm Rate: 31.91%
Confusion Matrix:
[[4821 2259]
 [2199 5001]]

Global Round: 35
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 35 Results:
Accuracy: 69.79%
Precision: 69.88%
Recall: 70.46%
F1 Score: 70.17%
AUC: 0.6978
False Alarm Rate: 30.89%
Confusion Matrix:
[[4893 2187]
 [2127 5073]]

Global Round: 36
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 36 Results:
Accuracy: 67.56%
Precision: 67.69%
Recall: 68.25%
F1 Score: 67.97%
AUC: 0.6756
False Alarm Rate: 33.14%
Confusion Matrix:
[[4734 2346]
 [2286 4914]]

Global Round: 37
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2


In [ ]:
# HFL 38 to 55

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
import os
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError as MSE

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
lr = 0.01

# Create metric instances for loading
mse_loss = MeanSquaredError()
mse_metric = MSE()

# Load the latest saved model
print("Loading latest model...")
global_model = load_model('/content/latest_model.h5',
                         custom_objects={
                             'MeanSquaredError': MeanSquaredError,
                             'mse': mse_loss,
                             'MSE': mse_metric
                         })

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Continue training
for comm_round in range(38, 55):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    global_model.save(f'/content/model_round_{comm_round}.h5')
    global_model.save('/content/latest_model.h5')

    # Model evaluation
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print(f"Round {comm_round} Results:")
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

# Save final model
global_model.save('/content/final_model.h5')

Loading latest model...



Global Round: 38
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 38 Results:
Accuracy: 72.55%
Precision: 72.59%
Recall: 73.19%
F1 Score: 72.89%
AUC: 0.7254
False Alarm Rate: 28.11%
Confusion Matrix:
[[5090 1990]
 [1930 5270]]

Global Round: 39
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 39 Results:
Accuracy: 76.34%
Precision: 76.32%
Recall: 76.96%
F1 Score: 76.64%
AUC: 0.7634
False Alarm Rate: 24.28%
Confusion Matrix:
[[5361 1719]
 [1659 5541]]

Global Round: 40
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Round 40 Results:
Accuracy: 65.80%
Precision: 65.95%
Recall: 66.50%
F1 Score: 66.22%
AUC: 0.6579
False Alarm Rate: 34.92%
Confusion Matrix:
[[4608 2472]
 [2412 4788]]

Global Round: 41
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Round 51 Results:
Accuracy: 68.22%
Precision: 68.33%
Recall: 68.90%
F1 Score: 68.62%
AUC: 0.6822
False Alarm Rate: 32.47%
Confusion Matrix:
[[4781 2299]
 [2239 4961]]

Global Round: 52
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 52 Results:
Accuracy: 71.90%
Precision: 71.96%
Recall: 72.56%
F1 Score: 72.25%
AUC: 0.7190
False Alarm Rate: 28.76%
Confusion Matrix:
[[5044 2036]
 [1976 5224]]

Global Round: 53
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 53 Results:
Accuracy: 70.76%
Precision: 70.83%
Recall: 71.42%
F1 Score: 71.12%
AUC: 0.7075
False Alarm Rate: 29.92%
Confusion Matrix:
[[4962 2118]
 [2058 5142]]

Global Round: 54
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Round 54 Results:
Accuracy: 69.51%
Precision: 69.60%
Recall: 70.18%
F1 Score: 69.89%
AUC: 0.6950
False Alarm Rate: 31.17%
Confusion Matrix:
[[4873 2207]
 [2147 5053]]


In [ ]:
# HFL 55 to 75

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
import os
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError as MSE

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
lr = 0.01

# Create metric instances for loading
mse_loss = MeanSquaredError()
mse_metric = MSE()

# Load the latest saved model
print("Loading latest model...")
global_model = load_model('/content/latest_model.h5',
                         custom_objects={
                             'MeanSquaredError': MeanSquaredError,
                             'mse': mse_loss,
                             'MSE': mse_metric
                         })

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Continue training
for comm_round in range(55, 75):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    global_model.save(f'/content/model_round_{comm_round}.h5')
    global_model.save('/content/latest_model.h5')

    # Model evaluation
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print(f"Round {comm_round} Results:")
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

# Save final model
global_model.save('/content/final_model.h5')

Loading latest model...

Global Round: 55
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Round 55 Results:
Accuracy: 75.52%
Precision: 75.51%
Recall: 76.14%
F1 Score: 75.82%
AUC: 0.7551
False Alarm Rate: 25.11%
Confusion Matrix:
[[5302 1778]
 [1718 5482]]

Global Round: 56
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 56 Results:
Accuracy: 74.96%
Precision: 74.96%
Recall: 75.58%
F1 Score: 75.27%
AUC: 0.7495
False Alarm Rate: 25.68%
Confusion Matrix:
[[5262 1818]
 [1758 5442]]

Global Round: 57
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 57 Results:
Accuracy: 76.83%
Precision: 76.80%
Recall: 77.44%
F1 Score: 77.12%
AUC: 0.7683
False Alarm Rate: 23.79%
Confusion Matrix:
[[5396 1684]
 [1624 5576]]

Global Round: 58
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 58 Results:
Accuracy: 75.90%
Precision: 75.88%
Recall: 76.51%
F1 Score: 76.20%
AUC: 0.7589
False Alarm Rate: 24.73%
Confusion Matrix:
[[5329 1751]
 [1691 5509]]

Global Round: 59
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 71 Results:
Accuracy: 79.08%
Precision: 79.01%
Recall: 79.67%
F1 Score: 79.34%
AUC: 0.7907
False Alarm Rate: 21.53%
Confusion Matrix:
[[5556 1524]
 [1464 5736]]

Global Round: 72
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 72 Results:
Accuracy: 78.00%
Precision: 77.95%
Recall: 78.60%
F1 Score: 78.27%
AUC: 0.7799
False Alarm Rate: 22.61%
Confusion Matrix:
[[5479 1601]
 [1541 5659]]

Global Round: 73
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 73 Results:
Accuracy: 81.93%
Precision: 81.82%
Recall: 82.50%
F1 Score: 82.16%
AUC: 0.8193
False Alarm Rate: 18.64%
Confusion Matrix:
[[5760 1320]
 [1260 5940]]

Global Round: 74
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Round 74 Results:
Accuracy: 80.48%
Precision: 80.39%
Recall: 81.06%
F1 Score: 80.72%
AUC: 0.8047
False Alarm Rate: 20.11%
Confusion Matrix:
[[5656 1424]
 [1364 5836]]


In [ ]:
# HFL 75 to 100

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
import os
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError as MSE

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def generate_prototypes(data, n_clusters=250):
    """Generate prototypes using KMeans"""
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data_scaled)
    return scaler.inverse_transform(kmeans.cluster_centers_)

# Initialize parameters
scaling_factor = 1
num_customers = 20
num_edge_servers = 4
clients_per_edge = num_customers // num_edge_servers
features = 48
lr = 0.01

# Create metric instances for loading
mse_loss = MeanSquaredError()
mse_metric = MSE()

# Load the latest saved model
print("Loading latest model...")
global_model = load_model('/content/latest_model.h5',
                         custom_objects={
                             'MeanSquaredError': MeanSquaredError,
                             'mse': mse_loss,
                             'MSE': mse_metric
                         })

# Load test data
test_features = pd.read_csv('/content/test_features.csv', sep=',', header=None)
test_labels = pd.read_csv('/content/test_labels.csv', sep=',', header=None)
X_test = test_features
y_test = test_labels.to_numpy()

# Continue training
for comm_round in range(75, 100):
    print(f"\nGlobal Round: {comm_round}")
    global_weights = global_model.get_weights()

    # Initialize edge-level aggregation
    edge_weights_list = []

    # Edge server loop
    for edge in range(num_edge_servers):
        print(f"Edge Server: {edge}")

        # Initialize weights for this edge server
        edge_local_weights = global_model.get_weights()
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] = 0

        # Client training within this edge server
        start_client = edge * clients_per_edge
        end_client = start_client + clients_per_edge

        for customer in range(start_client, end_client):
            print(f"Training Client: {customer}")

            # Load and process data
            train_features = pd.read_csv(f'/content/train_features{customer}.csv', sep=',', header=None)
            test_benign_features = pd.read_csv(f'/content/test_benign_features{customer}.csv', sep=',', header=None)

            # Generate prototypes
            xtr = generate_prototypes(train_features)
            xval = test_benign_features

            # Create and train local model
            local_model = Build_Autoencoder()
            local_model.compile(optimizer='adam', loss='mse')
            local_model.set_weights(global_weights)

            history = local_model.fit(xtr, xtr, epochs=70, batch_size=16, verbose=0, validation_data=(xval,xval))

            # Add scaled weights to edge aggregation
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            for i in range(len(scaled_weights)):
                for j in range(len(scaled_weights[i])):
                    edge_local_weights[i][j] += scaled_weights[i][j]

            tf.keras.backend.clear_session()

        # Average the weights for this edge server
        for i in range(len(edge_local_weights)):
            for j in range(len(edge_local_weights[i])):
                edge_local_weights[i][j] /= clients_per_edge

        edge_weights_list.append(edge_local_weights)

    # Global aggregation of edge servers
    global_weights = global_model.get_weights()
    for i in range(len(global_weights)):
        for j in range(len(global_weights[i])):
            global_weights[i][j] = sum(edge_weights[i][j] for edge_weights in edge_weights_list) / num_edge_servers

    # Update global model
    global_model.set_weights(global_weights)

    # Save model checkpoint
    global_model.save(f'/content/model_round_{comm_round}.h5')
    global_model.save('/content/latest_model.h5')

    # Model evaluation
    reconstructions = global_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

    clean = mse[np.where(y_test == 0)[0]]
    fraud = mse[np.where(y_test == 1)[0]]

    threshold = np.percentile(mse, 50 + ((len(clean)-len(fraud))/len(mse)*100))

    y_pred = np.ones(len(mse))
    for i in range(0, len(mse)):
        if mse[i] < threshold:
            y_pred[i] = 0

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    false_alarm_rate = fp / (fp + tn)

    print(f"Round {comm_round} Results:")
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    print("Precision: {:.2f}%".format(precision * 100))
    print("Recall: {:.2f}%".format(recall * 100))
    print("F1 Score: {:.2f}%".format(f1 * 100))
    print("AUC: {:.4f}".format(roc_auc))
    print("False Alarm Rate: {:.2f}%".format(false_alarm_rate * 100))
    print("Confusion Matrix:")
    print(cm)

# Save final model
global_model.save('/content/final_model.h5')

Loading latest model...

Global Round: 75
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Round 75 Results:
Accuracy: 81.20%
Precision: 81.10%
Recall: 81.78%
F1 Score: 81.44%
AUC: 0.8120
False Alarm Rate: 19.38%
Confusion Matrix:
[[5708 1372]
 [1312 5888]]

Global Round: 76
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 76 Results:
Accuracy: 82.51%
Precision: 82.38%
Recall: 83.07%
F1 Score: 82.72%
AUC: 0.8250
False Alarm Rate: 18.06%
Confusion Matrix:
[[5801 1279]
 [1219 5981]]

Global Round: 77
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 77 Results:
Accuracy: 79.94%
Precision: 79.86%
Recall: 80.53%
F1 Score: 80.19%
AUC: 0.7994
False Alarm Rate: 20.65%
Confusion Matrix:
[[5618 1462]
 [1402 5798]]

Global Round: 78
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 78 Results:
Accuracy: 79.47%
Precision: 79.39%
Recall: 80.06%
F1 Score: 79.72%
AUC: 0.7946
False Alarm Rate: 21.13%
Confusion Matrix:
[[5584 1496]
 [1436 5764]]

Global Round: 79
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


...

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 90 Results:
Accuracy: 81.55%
Precision: 81.45%
Recall: 82.12%
F1 Score: 81.78%
AUC: 0.8155
False Alarm Rate: 19.03%
Confusion Matrix:
[[5733 1347]
 [1287 5913]]

Global Round: 91
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 91 Results:
Accuracy: 82.35%
Precision: 82.23%
Recall: 82.92%
F1 Score: 82.57%
AUC: 0.8235
False Alarm Rate: 18.22%
Confusion Matrix:
[[5790 1290]
 [1230 5970]]

Global Round: 92
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 92 Results:
Accuracy: 80.24%
Precision: 80.15%
Recall: 80.82%
F1 Score: 80.48%
AUC: 0.8023
False Alarm Rate: 20.35%
Confusion Matrix:
[[5639 1441]
 [1381 5819]]

Global Round: 93
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 93 Results:
Accuracy: 80.85%
Precision: 80.76%
Recall: 81.43%
F1 Score: 81.09%
AUC: 0.8085
False Alarm Rate: 19.73%
Confusion Matrix:
[[5683 1397]
 [1337 5863]]

Global Round: 94
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
Training Client: 12
Training Client: 13
Training Client: 14
Edge Server: 3
Training Client: 15
Training Client: 16
Training Client: 17
Training Client: 18
Training Client: 19


447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Round 94 Results:
Accuracy: 80.14%
Precision: 80.06%
Recall: 80.72%
F1 Score: 80.39%
AUC: 0.8014
False Alarm Rate: 20.45%
Confusion Matrix:
[[5632 1448]
 [1388 5812]]

Global Round: 95
Edge Server: 0
Training Client: 0
Training Client: 1
Training Client: 2
Training Client: 3
Training Client: 4
Edge Server: 1
Training Client: 5
Training Client: 6
Training Client: 7
Training Client: 8
Training Client: 9
Edge Server: 2
Training Client: 10
Training Client: 11
